**GOLD LAYER**

In [0]:
# Gold Layer – Analytics & Aggregations
from pyspark.sql import functions as F
from pyspark.sql.functions import sum as spark_sum, avg,col


In [0]:
# Read silver layer Delta tables
df_school_master = spark.table("workspace.default.silver_school_master")
df_enrollment    = spark.table("workspace.default.silver_enrollment")
df_performance   = spark.table("workspace.default.silver_performance")

Aggregated enrollment metrics

In [0]:
#Remove records with invalid grade_level to ensure grade-wise analytics accuracy
df_enrollment_clean = df_enrollment.filter(
    col("grade_level") > 0
)


In [0]:
# Aggregate enrollment at school, year, grade, and gender level
from pyspark.sql.functions import sum as spark_sum

gold_enrollment_metrics= (
    df_enrollment_clean
    .groupBy("school_id", "academic_year", "grade_level", "gender")
    .agg(
        spark_sum("student_count").alias("total_students")
    )
) 

Aggregated performance & dropout metrics

In [0]:
from pyspark.sql.functions import avg,count

In [0]:
# Aggregate performance metrics at school-year-grade level
gold_performance_metrics = (
    df_performance
    .groupBy("school_id", "academic_year", "grade_level")
    .agg(
        avg("average_score").alias("avg_score"),
        avg("attendance_percentage").alias("avg_attendance"),
        spark_sum("dropout_risk_flag").alias("students_at_risk"),
        count("*").alias("total_students")
    )
)

In [0]:
from pyspark.sql.functions import when, col

In [0]:
# Calculate dropout percentage 
gold_performance_metrics = (
    gold_performance_metrics
    .withColumn(
        "dropout_risk_percent",
        when(
            col("total_students") > 0,
            (col("students_at_risk") / col("total_students")) * 100
        ).otherwise(0)
    )
)

In [0]:
gold_performance_metrics.select(
    "students_at_risk",
    "total_students",
    "dropout_risk_percent"
).show(10)

+----------------+--------------+--------------------+
|students_at_risk|total_students|dropout_risk_percent|
+----------------+--------------+--------------------+
|               2|             4|                50.0|
|               2|             2|               100.0|
|               3|             6|                50.0|
|               3|             6|                50.0|
|               2|             6|   33.33333333333333|
|               1|             2|                50.0|
|               1|             5|                20.0|
|               1|             3|   33.33333333333333|
|               0|             1|                 0.0|
|               2|             5|                40.0|
+----------------+--------------+--------------------+
only showing top 10 rows


Year-over-Year Enrollment Trends

In [0]:
from pyspark.sql.functions import sum as spark_sum

In [0]:
# Year-over-year enrollment trend at school level
enrollment_yoy = (
    gold_enrollment_metrics
    .groupBy("school_id", "academic_year")
    .agg(
        spark_sum("total_students").alias("yearly_enrollment")
    )
    .orderBy("school_id", "academic_year")
)

Gender & Grade-Level Distribution

In [0]:
# Aggregate enrollment by year and gender to analyze gender-wise distribution trends
gender_distribution = (
    gold_enrollment_metrics
    .groupBy("academic_year", "gender")
    .agg(
        spark_sum("total_students").alias("students_by_gender")
    )
)

In [0]:
# Aggregate enrollment by year and grade level to analyze grade-wise distribution trends
grade_distribution = (
    gold_enrollment_metrics
    .groupBy("academic_year", "grade_level")
    .agg(
        spark_sum("total_students").alias("students_by_grade")
    )
)

Dropout & Growth Patterns

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

In [0]:
high_dropout_risk = (
    gold_performance_metrics
    .filter(col("dropout_risk_percent") > 20)
)

In [0]:
# Define window for year-over-year comparison
window_spec = Window.partitionBy("school_id").orderBy("academic_year")

In [0]:
# Calculate year-over-year enrollment change using window functions
enrollment_growth = (
    enrollment_yoy
    .withColumn(
        "previous_year_enrollment",
        lag("yearly_enrollment").over(window_spec)
    )
    .withColumn(
        "enrollment_change",
        col("yearly_enrollment") - col("previous_year_enrollment")
    )
)

Regional & School-Level Comparisons

In [0]:
# Aggregate total enrollment by region and year for regional comparison analysis
regional_enrollment = (
    gold_enrollment_metrics
    .join(df_school_master, "school_id")
    .groupBy("region", "academic_year")
    .agg(
        spark_sum("total_students").alias("regional_enrollment")
    )
)

In [0]:
# Calculate region-wise dropout rate by aggregating at-risk and total students per year
regional_dropout = (
    gold_performance_metrics
    .join(df_school_master, "school_id")
    .groupBy("region", "academic_year")
    .agg(
        spark_sum("students_at_risk").alias("students_at_risk"),
        spark_sum("total_students").alias("total_students")
    )
    .withColumn(
        "regional_dropout_risk",
        (col("students_at_risk") / col("total_students")) * 100
    )
)


In [0]:
# Save final enrollment analytics table 
(
    gold_enrollment_metrics
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("enrollment_analytics")
)

In [0]:
# Save performance analytics table 
(
    gold_performance_metrics
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("performance_analytics")
)

In [0]:
%sql
SELECT * FROM workspace.default.performance_analytics

school_id,academic_year,grade_level,avg_score,avg_attendance,students_at_risk,total_students,dropout_risk_percent
SCH_0064,2019,-1,80.39333333333333,69.26666666666667,2,3,66.66666666666666
SCH_0040,2020,-1,52.36,87.3,0,1,0.0
SCH_0016,2020,-1,66.5725,83.52000000000001,1,4,25.0
SCH_0007,2019,-1,67.36999999999999,78.59750000000001,1,4,25.0
SCH_0107,2021,-1,66.52,85.03,0,1,0.0
SCH_0020,2024,-1,82.655,79.8,0,2,0.0
SCH_0039,2023,-1,70.94000000000001,82.32333333333334,1,3,33.33333333333333
SCH_0082,2019,-1,81.18333333333334,79.58333333333333,0,3,0.0
SCH_0102,2019,-1,62.21249999999999,79.7325,3,4,75.0
SCH_0064,2021,-1,52.636,81.556,3,5,60.0


**Validation(Airflow)**

In [0]:
quality_metrics = spark.sql("""
SELECT
    COUNT(*) AS total_records,
    SUM(CASE WHEN school_id IS NULL THEN 1 ELSE 0 END) AS null_school_ids,
    MIN(dropout_risk_percent) AS min_dropout,
    MAX(dropout_risk_percent) AS max_dropout
FROM performance_analytics
""")

quality_metrics.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("etl_quality_metrics")
